# Poly-Spline Finite-Element Method

Simple python notebook to regenrate the Fig 13.

The data and scripts to regenerate the other figures can be found 
[here](https://github.com/polyfem/Poly-Spline-Finite-Element-Method.git).

Note, we used [polyfem](https://polyfem.github.io) [C++ version](https://github.com/polyfem/polyfem/) for all figures.

Note that this can be run direcly with binder!
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/polyfem/Poly-Spline-Finite-Element-Method/master?filepath=Poly-Spline-Finite-Element-Method.ipynb)

Importing libraries

In [ ]:
import numpy as np
import polyfempy as pf
import json

#plotting:
import meshplot as mp

import platform

Note that this notebook is designed to be run in the root folder.
If you want to reproduce the results of the teaser you can find the images [here](https://github.com/polyfem/Decoupling-Simulation-Accuracy-from-Mesh-Quality/tree/master/fig1-teaser/meshes).

Setup problem

In [ ]:
#Uses laplacian
#we compare with Q2
settings = pf.Settings(
    pde=pf.PDEs.Laplacian,
    discr_order=2
)

if platform.system() == 'Windows':
    settings.set_advanced_option("solver_type", "Eigen::SparseLU")
else:
    settings.set_advanced_option("solver_type", "Eigen::UmfPackLU")


#and Franke function (https://github.com/polyfem/polyfem/blob/master/src/problem/FrankeProblem.cpp)
settings.problem = pf.Problems.Franke()

First we solve using standard $Q_2$ elements and our method which we enable with `settings.set_advanced_option("use_spline", True)`.

**Note 1** this takes some time.
**Note 2** on binder there is not enough memory, you can run it only in 2D, uncomment the second line.

In [ ]:
meshes = ["meshes/penguin.obj", "meshes/double_torus.HYBRID", "meshes/sculpt.HYBRID"]
# meshes = ["meshes/penguin.obj"]

total_solutions = {}

for use_spline in [False, True]:
    settings.set_advanced_option("use_spline", use_spline)
    
    solver = pf.Solver()
    solver.set_log_level(6)
    solver.settings(settings)

    solutions = []

    for mesh in meshes:
        #our method requires 1 level of refinement
        solver.load_mesh_from_path(mesh, normalize_mesh=True, n_refs=1)
        solver.solve()

        #getting and storing solution
        v, f, sol = solver.get_sampled_solution()
        solutions.append({"v": v, "f": f, "sol": sol})
        
    #store these list in the total one
    total_solutions["ours" if use_spline else "Q2"] = solutions

print("Done!")

It takes time, plz wait until done!

## Plot results

In [ ]:
#for nice plot we append the 2 meshses
def get_v_f_c(sols, i):
    #append points and offset x by 1.1
    v_ours = np.array(total_solutions["ours"][i]["v"])
    v_p1 = np.array(total_solutions["Q2"][i]["v"])
    v_p1[:, 0] += 1.1
    v = np.append(v_ours, v_p1, 0)
    
    #colors
    c_ours = np.array(total_solutions["ours"][i]["sol"])
    c_p1 = np.array(total_solutions["Q2"][i]["sol"])
    c = np.append(c_ours, c_p1, 0)
    
    #append faces
    f_ours = np.array(total_solutions["ours"][i]['f'])
    f_p1 = np.array(total_solutions["Q2"][i]['f'])
    f_p1 += np.max(f_ours)+1
    f = np.append(f_ours, f_p1, 0)
    
    return v, f, c

Our results are on the left!

In [ ]:
v, f, c = get_v_f_c(total_solutions, 0)
p = mp.plot(v, f, c, return_plot=True)

@mp.interact(mesh=[(meshes[i], i) for i in range(len(meshes))])
def ff(mesh):
    v, f, c = get_v_f_c(total_solutions, mesh)
    mp.plot(v, f, c, plot=p)
p